### Coletando notícias do G1

In [4]:
!pip install bs4

  Running setup.py bdist_wheel for bs4: started
  Running setup.py bdist_wheel for bs4: finished with status 'done'
  Stored in directory: C:\Users\Osvaldo Machado\AppData\Local\pip\Cache\wheels\a0\b0\b2\4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


You are using pip version 9.0.3, however version 18.0 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [6]:
!pip install selenium

You are using pip version 9.0.3, however version 18.0 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [9]:
from bs4 import BeautifulSoup
import time
from selenium import webdriver
import requests
import csv

## access given link and return corresponding new
def access_link(link):
    texto = ''
    page = requests.get(link)
    soup = BeautifulSoup(page.content, 'html.parser')
    paragrafos = soup.find_all(class_="content-text__container")
    for par in paragrafos:
        texto = texto + '\n' + par.getText()
    title = soup.find(class_="content-head__title")
    return title.getText(), texto

links = ['https://globoesporte.globo.com/numerologos/noticia/flamengo-tem-sua-maior-media-de-publico-no-brasileirao-dos-pontos-corridos.ghtml',
        'https://g1.globo.com/ciencia-e-saude/noticia/2018/08/11/nasa-lanca-missao-para-tocar-o-sol-com-nave-hiper-resistente-ao-calor-e-a-radiacao.ghtml',
        'https://g1.globo.com/mg/triangulo-mineiro/noticia/2018/08/10/prefeitura-de-araguari-escalona-salario-de-servidores.ghtml',
        'https://g1.globo.com/mg/triangulo-mineiro/noticia/2018/08/09/porco-espinho-e-retirado-de-cima-de-arvore-sibipiruna-em-araguari.ghtml']

news_data = []
for link in links:
    title, texto = access_link(link)
    news_data.append({'title': title, 'content': texto})

for news in news_data:
    print(news['title'], '\n\n', news['content'], '\n\n\n')

Flamengo tem sua maior média de público no Brasileirão dos pontos corridos 

 
 A ótima campanha do Flamengo no Campeonato Brasileiro tem reflexo direto nas arquibancadas. Nenhum time levou tantos torcedores aos estádios na Série A quanto o Rubro-Negro. Em nove jogos como mandante, todos no Maracanã, o Fla tem média de público de 47.596 pagantes, a maior do clube na era dos pontos corridos, superando a marca de 2008, que registrou 40.694 torcedores por partida. 
 A média atual do time carioca impressiona tanto que a diferença para o São Paulo, segundo colocado neste levantamento, é de 15.529 pagantes. E esses números podem subir mais no domingo. O clube divulgou na noite da última sexta-feira que já foram vendidos 45 mil ingressos para o duelo diante do Cruzeiro, às 16h, no Maracanã. 
 Confira os públicos do Flamengo em 2018 
 O maior público do Flamengo como mandante no Brasileirão foi na vitória sobre o Internacional, que teve 55.283 ingressos vendidos, o terceiro maior da competição

### Calculando estatísticas básicas de cada notícia coletada (news_data)

In [14]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Osvaldo
[nltk_data]     Machado\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [15]:
import sys
import json
import nltk



stop_words = nltk.corpus.stopwords.words('portuguese') + [
    '.',
    ',',
    '--',
    '\'s',
    '?',
    ')',
    '(',
    ':',
    '\'',
    '\'re',
    '"',
    '-',
    '}',
    '{',
    ]

for news in news_data:
    sentences = nltk.tokenize.sent_tokenize(news['content'])

    words = [w.lower() for sentence in sentences for w in
             nltk.tokenize.word_tokenize(sentence)]

    fdist = nltk.FreqDist(words)

    # Basic stats

    num_words = sum([i[1] for i in fdist.items()])
    num_unique_words = len(fdist.keys())

    # Hapaxes are words that appear only once

    num_hapaxes = len(fdist.hapaxes())

    top_10_words_sans_stop_words = [w for w in fdist.items() if w[0]
                                    not in stop_words][:10]

    print(news['title'])
    print('\tNum Sentences:'.ljust(25), len(sentences))
    print('\tNum Words:'.ljust(25), num_words)
    print('\tNum Unique Words:'.ljust(25), num_unique_words)
    print('\tNum Hapaxes:'.ljust(25), num_hapaxes)
    print('\tTop 10 Most Frequent Words (sans stop words):\n\t\t', \
            '\n\t\t'.join(['%s (%s)'
            % (w[0], w[1]) for w in top_10_words_sans_stop_words]))
    print()

Flamengo tem sua maior média de público no Brasileirão dos pontos corridos
	Num Sentences:           9
	Num Words:               268
	Num Unique Words:        155
	Num Hapaxes:             122
	Top 10 Most Frequent Words (sans stop words):
		 ótima (1)
		campanha (1)
		flamengo (3)
		campeonato (1)
		brasileiro (1)
		reflexo (1)
		direto (1)
		arquibancadas (1)
		nenhum (1)
		time (2)

Nasa adia lançamento de missão para 'tocar' o Sol com nave hiper-resistente ao calor e à radiação
	Num Sentences:           54
	Num Words:               1071
	Num Unique Words:        396
	Num Hapaxes:             258
	Top 10 Most Frequent Words (sans stop words):
		 agência (2)
		espacial (3)
		americana (1)
		nasa (4)
		adiou (1)
		terceira (1)
		vez (3)
		lançamento (3)
		primeira (2)
		missão (7)

Prefeitura de Araguari escalona salário de servidores
	Num Sentences:           15
	Num Words:               412
	Num Unique Words:        195
	Num Hapaxes:             129
	Top 10 Most Frequent Words (sans

### Sumarizando as notícias coletadas

In [16]:
import sys
import json
import nltk
import numpy

N = 100  # Number of words to consider
CLUSTER_THRESHOLD = 5  # Distance between words to consider
TOP_SENTENCES = 5  # Number of sentences to return for a "top n" summary

# Approach taken from "The Automatic Creation of Literature Abstracts" by H.P. Luhn

def _score_sentences(sentences, important_words):
    scores = []
    sentence_idx = -1

    for s in [nltk.tokenize.word_tokenize(s) for s in sentences]:

        sentence_idx += 1
        word_idx = []

        # For each word in the word list...
        for w in important_words:
            try:
                # Compute an index for where any important words occur in the sentence

                word_idx.append(s.index(w))
            except (ValueError) as e: # w not in this particular sentence
                pass

        word_idx.sort()

        # It is possible that some sentences may not contain any important words at all
        if len(word_idx)== 0: continue

        # Using the word index, compute clusters by using a max distance threshold
        # for any two consecutive words

        clusters = []
        cluster = [word_idx[0]]
        i = 1
        while i < len(word_idx):
            if word_idx[i] - word_idx[i - 1] < CLUSTER_THRESHOLD:
                cluster.append(word_idx[i])
            else:
                clusters.append(cluster[:])
                cluster = [word_idx[i]]
            i += 1
        clusters.append(cluster)

        # Score each cluster. The max score for any given cluster is the score 
        # for the sentence

        max_cluster_score = 0
        for c in clusters:
            significant_words_in_cluster = len(c)
            total_words_in_cluster = c[-1] - c[0] + 1
            score = 1.0 * significant_words_in_cluster \
                * significant_words_in_cluster / total_words_in_cluster

            if score > max_cluster_score:
                max_cluster_score = score

        scores.append((sentence_idx, score))

    return scores

def summarize(txt):
    sentences = [s for s in nltk.tokenize.sent_tokenize(txt)]
    normalized_sentences = [s.lower() for s in sentences]

    words = [w.lower() for sentence in normalized_sentences for w in
             nltk.tokenize.word_tokenize(sentence)]

    fdist = nltk.FreqDist(words)

    top_n_words = [w[0] for w in fdist.items() 
            if w[0] not in nltk.corpus.stopwords.words('portuguese')][:N]

    scored_sentences = _score_sentences(normalized_sentences, top_n_words)

    # Summaization Approach 1:
    # Filter out non-significant sentences by using the average score plus a
    # fraction of the std dev as a filter

    avg = numpy.mean([s[1] for s in scored_sentences])
    std = numpy.std([s[1] for s in scored_sentences])
    mean_scored = [(sent_idx, score) for (sent_idx, score) in scored_sentences
                   if score > avg + 0.5 * std]

    # Summarization Approach 2:
    # Another approach would be to return only the top N ranked sentences

    top_n_scored = sorted(scored_sentences, key=lambda s: s[1])[-TOP_SENTENCES:]
    top_n_scored = sorted(top_n_scored, key=lambda s: s[0])

    # Decorate the post object with summaries

    return dict(top_n_summary=[sentences[idx] for (idx, score) in top_n_scored],
                mean_scored_summary=[sentences[idx] for (idx, score) in mean_scored])


if __name__ == '__main__':

    # Load in output from blogs_and_nlp__get_feed.py

    #BLOG_DATA = sys.argv[1]
    #blog_data = json.loads(open(BLOG_DATA).read())
    
    for post in news_data:
       
        post.update(summarize(post['content']))

        print(post['title'])
        print('-' * len(post['title']))
        print()
        print('-------------')
        print('Top N Summary')
        print('-------------')
        print(' '.join(post['top_n_summary']))
        print()
        print('-------------------')
        print('Mean Scored Summary')
        print('-------------------')
        print(' '.join(post['mean_scored_summary']))
        print()

Flamengo tem sua maior média de público no Brasileirão dos pontos corridos
--------------------------------------------------------------------------

-------------
Top N Summary
-------------
Em nove jogos como mandante, todos no Maracanã, o Fla tem média de público de 47.596 pagantes, a maior do clube na era dos pontos corridos, superando a marca de 2008, que registrou 40.694 torcedores por partida. A média atual do time carioca impressiona tanto que a diferença para o São Paulo, segundo colocado neste levantamento, é de 15.529 pagantes. O clube divulgou na noite da última sexta-feira que já foram vendidos 45 mil ingressos para o duelo diante do Cruzeiro, às 16h, no Maracanã. Confira os públicos do Flamengo em 2018 
 O maior público do Flamengo como mandante no Brasileirão foi na vitória sobre o Internacional, que teve 55.283 ingressos vendidos, o terceiro maior da competição. O recorde até agora foi no Fla-Flu no Mané Garrincha, com 59.987 pagantes, que, apesar da maioria no estádio

### Visualização HTML da sumarização

In [17]:
import os
import sys
import json
import nltk
import numpy

HTML_TEMPLATE = """<html>
    <head>
        <title>%s</title>
        <meta http-equiv="Content-Type" content="text/html; charset=UTF-8"/>
    </head>
    <body>%s</body>
</html>"""

if __name__ == '__main__':

    # Marked up version can be written out to disk

    if not os.path.isdir('out/summarize'):
        os.makedirs('out/summarize')

    for post in news_data:
       
        post.update(summarize(post['content']))

        for summary_type in ['top_n_summary', 'mean_scored_summary']:
            post[summary_type + '_marked_up'] = '<p>%s</p>' % (post['content'], )
            for s in post[summary_type]:
                post[summary_type + '_marked_up'] = \
                post[summary_type + '_marked_up'].replace(s, '<strong>%s</strong>' % (s, ))

            filename = post['title'] + '.summary.' + summary_type + '.html'
            f = open(os.path.join('out', 'summarize', filename), 'w')
            html = HTML_TEMPLATE % (post['title'] + ' Summary', post[summary_type + '_marked_up'],)
            #f.write(str(html.encode('utf-8')))
            f.write(html)
            f.close()

            print("Data written to", f.name)

Data written to out\summarize\Flamengo tem sua maior média de público no Brasileirão dos pontos corridos.summary.top_n_summary.html
Data written to out\summarize\Flamengo tem sua maior média de público no Brasileirão dos pontos corridos.summary.mean_scored_summary.html
Data written to out\summarize\Nasa adia lançamento de missão para 'tocar' o Sol com nave hiper-resistente ao calor e à radiação.summary.top_n_summary.html
Data written to out\summarize\Nasa adia lançamento de missão para 'tocar' o Sol com nave hiper-resistente ao calor e à radiação.summary.mean_scored_summary.html
Data written to out\summarize\Prefeitura de Araguari escalona salário de servidores.summary.top_n_summary.html
Data written to out\summarize\Prefeitura de Araguari escalona salário de servidores.summary.mean_scored_summary.html
Data written to out\summarize\Porco-espinho é retirado de cima de árvore sibipiruna em Araguari .summary.top_n_summary.html
Data written to out\summarize\Porco-espinho é retirado de cima